# Assignment 1/2

What is the price of a European call option on a non-dividend-paying stock when the stock price is $\$52$, the strike is $\$50$, the risk-free interest rate is $12\%$ per annum, the volatility is $30\%$ per annum, the time to maturity is $3$ months?

- Build and use a binomial tree with
    - 1 step
    - 2 steps
    - 3 steps
    - n steps to calculate approximate option prices. Compare to the formula result in Assignment 1/3. Analyze the scaling to the Black-Scholes formula result. 

Use Python and send your Jupyter notebook containing your results (further assignments will use this notebook).

## Solution

Evaluation of the price of an option with binomial trees is a three step process:

1. Creation of the binomial price tree
2. Calculation of option value at each final node
3. Sequential calculation of the option value at each preceding node

#### Source
https://en.wikipedia.org/wiki/Binomial_options_pricing_model

In [ ]:
import numpy as np

import seaborn as sns
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt

#### Just some matplotlib and seaborn parameter tuning

In [ ]:
# Set axtick dimensions
major_size = 6
major_width = 1.2
minor_size = 3
minor_width = 1
mpl.rcParams['xtick.major.size'] = major_size
mpl.rcParams['xtick.major.width'] = major_width
mpl.rcParams['xtick.minor.size'] = minor_size
mpl.rcParams['xtick.minor.width'] = minor_width
mpl.rcParams['ytick.major.size'] = major_size
mpl.rcParams['ytick.major.width'] = major_width
mpl.rcParams['ytick.minor.size'] = minor_size
mpl.rcParams['ytick.minor.width'] = minor_width

# Seaborn style settings
sns.set_style({'axes.axisbelow': True,
               'axes.edgecolor': '.1',
               'axes.facecolor': 'white',
               'axes.grid': True,
               'axes.labelcolor': '.15',
               'axes.spines.bottom': True,
               'axes.spines.left': True,
               'axes.spines.right': True,
               'axes.spines.top': True,
               'figure.facecolor': 'white',
               'font.family': ['sans-serif'],
               'font.sans-serif': ['Arial',
                'DejaVu Sans',
                'Liberation Sans',
                'Bitstream Vera Sans',
                'sans-serif'],
               'grid.color': '.8',
               'grid.linestyle': '--',
               'image.cmap': 'rocket',
               'lines.solid_capstyle': 'round',
               'patch.edgecolor': 'w',
               'patch.force_edgecolor': True,
               'text.color': '.15',
               'xtick.bottom': True,
               'xtick.color': '.15',
               'xtick.direction': 'in',
               'xtick.top': True,
               'ytick.color': '.15',
               'ytick.direction': 'in',
               'ytick.left': True,
               'ytick.right': True})

In [ ]:
def euro_option_binomial_price(S_t, K, T, r, sigma, N):

    # Step 1.
    # Creation of the price tree
    C = np.zeros((N+1, N+1))    # Option value for the ith node at time t
    S = np.zeros((N+1, N+1))    # Spot prices of the underlying asset at the nth period 
    
    # Each layer in the tree represents a time step
    dT = T / N

    # Up and down factors
    # The price goes up or down in each step
    # d = 1/u
    u = np.exp(sigma * np.sqrt(dT))
    d = np.exp(-sigma * np.sqrt(dT))
    
    # Binomial prob.
    p = (np.exp(r * dT) - d) / (u - d)
    
    # P up and down
    P_u = np.exp(-r * dT) * p
    P_d = np.exp(-r * dT) * (1 - p)
    
    # Step 2.
    # Find the option value at each final node
    for i in range(0, N+1):
        for j in range(i, N+1):
            S[i, j] = S_t * u**j * d**(2*i)
    for i in range(0, N+1):
        # numpy.max() does not work here...?
        C[i, N] = max(0, S[i, N] - K)
    
    # Step 3.
    # Calculate the option values at each preceding node
    for j in range(N-1, -1, -1):
        for i in range(0, j+1):
            C[i, j] = P_u * C[i, j+1] + P_d * C[i+1, j+1]

    return C[0, 0]

In [ ]:
S_t = 52
K = 50
T = 3/12
r = 0.12
sigma = 0.3

In [ ]:
euro_option_binomial_price(S_t, K, T, r, sigma, N=1)

In [ ]:
euro_option_binomial_price(S_t, K, T, r, sigma, N=2)

In [ ]:
euro_option_binomial_price(S_t, K, T, r, sigma, N=3)

In [ ]:
euro_option_binomial_price(S_t, K, T, r, sigma, N=50)

### N steps

In [ ]:
bs_value = 5.0574

N_values = np.arange(1,101,1)
price_approx = [euro_option_binomial_price(S_t, K, T, r, sigma, N=n) for n in N_values]

In [ ]:
nrows = 1
ncols = 1
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols*10, nrows*8))

axislabelsize = 18
axisticksize = 15
axislegendsize = 18

axes.plot(N_values, price_approx, label='Binomial tree model',
          c='tab:red', lw=3)
axes.axhline(y=bs_value, label='Black-Scholes model',
             c='green', lw=3, ls='--', alpha=0.7)


axes.set_xlabel('$N$', fontsize=axislabelsize)
axes.set_ylabel('Price of option [USD]', fontsize=axislabelsize)
axes.tick_params(axis='both', which='major', labelsize=axisticksize)

axes.legend(loc='upper right', fontsize=axislegendsize)

plt.show()

In [ ]:
print('At N = {0}, the ratio of the Binomial approximation and the B-S formula is {1:.5f}'.format(np.max(N_values),
                                                                                                  price_approx[-1]/bs_value))

We can state, that the method involving a binomial tree, approximates the result given by the Black-Scholes model.